# Evaluation and Comparison of Boosted ML Models in Behavior-Based Malware Detection


## Notebook: CatBoost Tuning

***

**What is the objective of this file?**

To tune the model using the Train Split (Dynamically Split).

# Checklist

- Ensure that you have the proper dataset files that you intend to use (i.e., whether the lite dataset or full version). 
    - The datasets it will use points to `/Official Development/Dataset/IB` & `/Official Development/Dataset/TB`. 
    - You can run the `/Official Development/Dataset [OFFICIAL] Oliveira Dataset Notebook.ipynb file` or unzip one of the zipped folders in the `/Official Development/Dataset/Processed` towards the two aforementioned folders. 
- Ensure that you have installed the necessary libraries needed to execute the training process. 
    - You can view the list of the specific versions in the thesis document or through the `.sh` or `.bat` files in the repository's home directory.

# 1. CatBoost Tuning Setup

Setting tuning environment parameters.

## 1.0. Tuning Settings

1. What will the output filename be?
2. What is the dictionary of hyperparameter values?

In [ ]:
OUTPUT_FILENAME = "TEST_TUNE" # <== Specify the filename of the output hyperparameters text file (can be used during tuned model training).

K = 5
DYNAMIC_SPLIT = 1/K # <== Expected validation ratio. Training ratio will be 1-(1/K). Just for when needed to view.
RANDOM_SEED = 1 # <==  <== Must be the same throughout the entire study (acts as a controlled variable), hence let's just settle with 1.

**Recommended Parameters as per the paper:**
1. grow_policy = SymmetricTree
2. objective = Logloss
3. bootstrap_type* = Bayesian
4. boosting_type = Ordered

*As per documentation relative to study's objectives and needs.

In [ ]:
#Sample (tested) parameter set
catb_params = {
    #Based-on Study/Documentation (Do not change)
    'grow_policy':['SymmetricTree'], #By design, CatBoost is intended to grow symmetrically.
    'objective':['Logloss'], #To be same as what's used in Binary LightGBMClassifier
    'bootstrap_type':['Bayesian'], #Recommended for classifier models
    'boosting_type':['Ordered'], #As per paper "Ordered Boosting"
    
    #Common (Add and experiment with valid values)
    'max_depth':[6,16], #As per documentation: min=6, max=16 (Logloss), mean=(max-min)/2
    'learning_rate':[0.03, 0.1, 0.3], #Typical learning rate (either 0.01, 0.03, 0.1, 0.3)
    'n_estimators':[10,25,50,75,100,250,500,750,1000], #Limited to 50 for prototyping; [50,100,250,500,750,1000,2000,5000,10000]
    'l2_leaf_reg':[3], #CatB default=3
    'auto_class_weights':['None', 'Balanced', 'SqrtBalanced'], #Specify that classes are weighted equally (i.e., no inherent bias); ['None', 'Balanced', 'SqrtBalanced']
    
    #Other hyperparameters
    
    #Others; Not common but can improve accuracy/performance:
    'task_type':['CPU'], #CPU/GPU (CUDA)
}

# ⚠️Warning

**Be careful of modifying the code beyond this point as it was designed to run autonomously based on the parameters set above.**

## 1.1. Loading Libraries & Diagnostics

In [ ]:
#Python Libraries
import time
from datetime import datetime
import os

#Data/Dataset Libraries
import pandas as pd
import numpy as np

#Model Selection
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#Metrics (for in-tuning testing only)
from sklearn.metrics import classification_report, accuracy_score, balanced_accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score

#Visualization
from matplotlib import pyplot as plt

#GBDT Models
# import lightgbm
import catboost

#File Writing Library (exclusive for use on LightGBM)
from joblib import dump, load

start = end = 0
LOG_FILENAME = "CATB_Tuning_Log.txt"
def logging(message):
    log = open(LOG_FILENAME, "a")
    log.write(message)
    log.close()
def startTime():
    global start
    start = time.time()
def endTime(process):
    global start
    elapse = time.time()-start
    start = 0
    printout = f"{str(datetime.now())}@{OUTPUT_FILENAME}: {process} - {round(elapse, 6)}s\n"
    logging(printout)
    return round(elapse, 6)
def printToFile(label, output):
    global OUTPUT_FILENAME
    filename = OUTPUT_FILENAME + "_" + label + ".tune"
    with open("Outputs/" + filename, 'w') as f:
        f.write(str(output))
        f.close()
def getIndexes(dataset):
    indexes = []
    for i in range(dataset.shape[1]-1):
        indexes.append("t_"+str(i))
    return indexes

## 1.2. Loading Datasets

### 1.2.1. Setting Filenames

In [ ]:
#Setting filenames of files
TB_Train = "../Dataset/TB/TB_CATB.csv" # <== Location for Time-based Train Split for LightGBM
#TB_Test = "../Dataset/TB/TB_Test_CATB.csv" # <== Location for Time-based Test Split for LightGBM
IB_Train = "../Dataset/IB/IB_CATB.csv" # <== Location for Instance-based Train Split for LightGBM
#IB_Test = "../Dataset/IB/IB_Test_CATB.csv" # <== Location for Instance-based Test Split for LightGBM

### 1.2.2. Loading datasets to DataFrames

In [ ]:
#Loading datasets to DataFrames
tb_train = pd.read_csv(TB_Train, low_memory=False).fillna("NaN")
ib_train = pd.read_csv(IB_Train, low_memory=False).fillna("NaN")

print("Dataset Sizes")
print("TB Train Size:", tb_train.shape)
print("IB Train Size:", ib_train.shape)

### 1.2.3. Previewing datasets

In [ ]:
#Previewing Time-based Dataset
tb_train.head()

In [ ]:
#Previewing Instance-based Dataset
ib_train.head()

## 1.3. Preparing Stratified K-Folds for RandomizedSearchCV

By default, RandomizedSearchCV uses a normal K-Folds with k value of 5.

### 1.3.1. Underlying Code for Stratified K-Folds

Most of the code is for visualizing actually. Only `getStratKFold()` matters.

In [ ]:
def getStratKFold():
    global K
    return StratifiedKFold(n_splits=K, shuffle=True, random_state=1)

#K-folds sample visualization
def kfolds_vis(dataset):
    global K
    X = dataset.iloc[:,1:] #All rows, 2nd to last column
    y = dataset.iloc[:,0] #All rows, first column only
    fig, ax = plt.subplots(figsize=(10,K+1))
    train = plot_cv_indices(getStratKFold(), X, y, ax, K)
    plt.show()

#K-folds sample visualization (inner workings)
def plot_cv_indices(cv, X, y, ax, n_splits, lw=25):
    #From: https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y)):
        indices = np.array([np.nan] * len(X))
        indices[tt], indices[tr] = 1,0
        ax.scatter(range(len(indices)), [ii] * len(indices), c=indices, marker="_", 
                   lw=lw, cmap=plt.cm.Paired, vmin=0, vmax=1)
    #ax.scatter(range(len(X)), [ii] * len(X), c=y, marker="_", lw=lw, cmap=plt.cm.Paired)
    yticklabels = list(range(n_splits))
    ax.set(yticks=np.arange(n_splits), yticklabels=yticklabels, xlabel="Dataset Subsample", ylabel="CV iteration", ylim=[n_splits,-1], xlim=[0, X.shape[0]])
    ax.set_title("{}".format(type(cv).__name__))
    return ax

### 1.3.2. Visualizing the K-Folds

The objective of the visualization is to show that no CV iteration will have the same set of samples (i.e., no overfitting).

In [ ]:
print("Stratified K-Folds Split at",K,"splits.")
print("TB/TB_Encoded Dataset")
kfolds_vis(tb_train)
print("IB/IB_Encoded Dataset")
kfolds_vis(ib_train)

### 1.3.3. Plotting Search CV Results

As suggested from https://stackoverflow.com/a/57013458

In [ ]:
def plot_search_results(grid, cv_results, scoring=['Accuracy','Precision','Recall']):
    ## Results from grid search
    results = grid.cv_results_
    means_test_acc = results['mean_test_accuracy']
    stds_test_acc = results['std_test_accuracy']
    # means_train_acc = results['mean_train_accuracy']
    # stds_train_acc = results['std_train_accuracy']
    means_test_prec = results['mean_test_precision']
    stds_test_prec = results['std_test_precision']
    # means_train_prec = results['mean_train_precision']
    # stds_train_prec = results['std_train_precision']
    means_test_rec = results['mean_test_recall']
    stds_test_rec = results['std_test_recall']
    # means_train_rec = results['mean_train_recall']
    # stds_train_rec = results['std_train_recall']
    ## Getting indexes of values per hyper-parameter
    masks=[]
    masks_names= list(grid.best_params_.keys())
    for p_k, p_v in grid.best_params_.items():
        masks.append(list(results['param_'+p_k].data==p_v))
    params = grid.get_params()['param_grid']
    ## Ploting results
    fig, ax = plt.subplots(int(len(params)/4),4, sharex='none', sharey='none',figsize=(16,9))
    fig.suptitle(f'Score per parameter {str(scoring)}')
    fig.text(0.04, 0.5, 'MEAN SCORE', va='center', rotation='vertical')
    pram_preformace_in_best = {}
    row = col = 0
    for i, p in enumerate(masks_names):
        m = np.stack(masks[:i] + masks[i+1:])
        pram_preformace_in_best
        best_parms_mask = m.all(axis=0)
        best_index = np.where(best_parms_mask)[0]
        x = np.array(params[p])
        ax[row,col].errorbar(x, np.array(means_test_acc[best_index]), np.array(stds_test_acc[best_index]), linestyle='--', marker='o', label='Accuracy (Test)')
        # ax[row,col].errorbar(x, np.array(means_train_acc[best_index]), np.array(stds_train_acc[best_index]), linestyle='-', marker='^',label='Accuracy (Train)')
        ax[row,col].errorbar(x, np.array(means_test_prec[best_index]), np.array(stds_test_prec[best_index]), linestyle='--', marker='o', label='Precision (Test)')
        # ax[row,col].errorbar(x, np.array(means_train_prec[best_index]), np.array(stds_train_prec[best_index]), linestyle='-', marker='^',label='Precision (Train)')
        ax[row,col].errorbar(x, np.array(means_test_rec[best_index]), np.array(stds_test_rec[best_index]), linestyle='--', marker='o', label='Recall (Test)')
        # ax[row,col].errorbar(x, np.array(means_train_rec[best_index]), np.array(stds_train_rec[best_index]), linestyle='-', marker='^',label='Recall (Train)')
        ax[row,col].set_xlabel(p.upper())
        col += 1
        if col >= 4:
            row += 1
            col = 0
    plt.legend(loc='upper left')
    plt.show()
    plt.figure(figsize=(16,9))
    # plt.plot(cv_results.index, cv_results['mean_train_accuracy'], linestyle='-', marker='o', label='Accuracy (Train)')
    plt.plot(cv_results.index, cv_results['mean_test_accuracy'], linestyle='-', marker='o', label='Accuracy (Test)')
    # plt.plot(cv_results.index, cv_results['mean_train_precision'], linestyle='-', marker='o', label='Precision (Train)')
    plt.plot(cv_results.index, cv_results['mean_test_precision'], linestyle='-', marker='o', label='Precision (Test)')
    # plt.plot(cv_results.index, cv_results['mean_train_recall'], linestyle='-', marker='o', label='Recall (Train)')
    plt.plot(cv_results.index, cv_results['mean_test_recall'], linestyle='-', marker='o', label='Recall (Test)')
    plt.ylabel('Score')
    plt.xlabel('Iterations')
    plt.title(f'{str(scoring)} per Iteration')
    plt.legend(loc='upper left')
    plt.show()

## 2. Tuning Proper

*Actual Model Tuning Process*

**Notes:**
1. Not all hyperparameters are compatible with each other.
2. Select, test, and prioritize ones that are relevant to the project.
3. The example shown below are already tested for compatibility. 
4. Watch your terminal for outputs.

LGBM Hyperparameters: https://lightgbm.readthedocs.io/en/latest/Parameters.html

CatBoost Hyperparameters: https://catboost.ai/en/docs/references/training-parameters/

## 2.1. Tuning Preparation

In [ ]:
#Create the Stratified K-Folds
stratKFold = getStratKFold()

#Specify features (X) and labels (y) for Time-based data
X_tb = tb_train.iloc[:,1:] #All rows, 2nd to last column
y_tb = tb_train.iloc[:,0] #All rows, first column only

#Specify features (X) and labels (y) for Instance-based data
X_ib = ib_train.iloc[:,1:] #All rows, 2nd to last column
y_ib = ib_train.iloc[:,0] #All rows, first column only

def get_threshold(cv_results, target:str):
    return cv_results[target].max() - cv_results[target].std()

def refit_strategy(cv_results):
    cv_results_ = pd.DataFrame(cv_results)
    # Filter-out all results below the threshold
    high_accuracy_cv_results = cv_results_[cv_results_["mean_test_accuracy"] > .90]
    high_precision_cv_results = high_accuracy_cv_results[high_accuracy_cv_results["mean_test_precision"] > get_threshold(high_accuracy_cv_results, 'mean_test_precision')]
    high_recall_cv_results = high_precision_cv_results[high_precision_cv_results["mean_test_recall"] > get_threshold(high_precision_cv_results, 'mean_test_recall')]

    return high_recall_cv_results["mean_score_time"].idxmin() #Return the one that is fastest

def getTuner(model, params, scoring=['accuracy','precision','recall']):
    return GridSearchCV(model, params, scoring=scoring, n_jobs=1, refit=refit_strategy, cv=getStratKFold(), verbose=2, pre_dispatch='2*n_jobs', error_score=0, return_train_score=False)

## 2.2. Time-based Model Tuning

In [ ]:
startTime()
catb_classifier = catboost.CatBoostClassifier(random_state=1, thread_count=os.cpu_count(), verbose=1, cat_features=getIndexes(tb_train), nan_mode='Min')
tb_tuner = getTuner(catb_classifier, catb_params)
tb_tuner.fit(X_tb, y_tb)
endTime("TB_CATB")

printToFile("TB_CATB", tb_tuner.best_params_)
cv_results = pd.DataFrame.from_dict(tb_tuner.cv_results_)
cv_results.to_csv('CATB_TB_Tuning_CV_Results.csv')
plot_search_results(tb_tuner, cv_results)
cv_results

In [ ]:
plot_search_results(tb_tuner)

## 2.3. Instance-based Model Tuning

In [ ]:
startTime()
catb_classifier = catboost.CatBoostClassifier(random_state=1, thread_count=os.cpu_count(), verbose=0, cat_features=getIndexes(tb_train), nan_mode='Min')
ib_tuner = getTuner(catb_classifier, catb_params)
ib_tuner.fit(X_ib, y_ib)
endTime("IB_CATB")

printToFile("IB_CATB", tb_tuner.best_params_)
cv_results = pd.DataFrame.from_dict(tb_tuner.cv_results_)
cv_results.to_csv('CATB_IB_Tuning_CV_Results.csv')
plot_search_results(tb_tuner, cv_results)
cv_results

In [ ]:
logging("\n")